In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

/usr/local/lib/python3.5/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
acc_cas = {}
for i in range(2010, 2017):
    idx = str(i)
    acc = pd.read_excel('./input/{0}/Acc_cas {0}.xls'.format(idx))
    acc = acc[['ACC_TIME', 'WEEK_DAY', 'RAIN', 'SPEED_LMT', 'TRAFF_CONG', 'JCN_CTRL',
               'RD_TYPE', 'CAS_AGE', 'CAS_SEX', 'INJURY', 'ROLE', 'SB_WORN', 'SEAT', 
               'PED_LOCATN']]

    acc['YEAR'] = idx
    acc_cas[idx] = acc
    print('The Year %s is read.' % idx)

The Year 2010 is read.
The Year 2011 is read.
The Year 2012 is read.
The Year 2013 is read.
The Year 2014 is read.
The Year 2015 is read.
The Year 2016 is read.


In [4]:
acc = acc_cas['2010'].copy()
for i in range(2010, 2017):
    acc = acc.append(acc_cas[str(i)].copy())

In [5]:
def str_append(x):
    if isinstance(x, str):
        counter = ['0' for i in range(4 - len(x))]
        counter = ''.join(counter)
        x = counter + x
        return x
    else:
        raise TypeError('The function only support string-type input.')


acc['ACC_TIME'] = acc['ACC_TIME'].map(
    lambda x: str_append(str(x)))
acc['HOUR'] = pd.to_datetime(
    acc['ACC_TIME'], format='%H%M').dt.hour
t0 = acc['HOUR'][acc['HOUR'].isin(
    [7, 8, 9])].count()  # 700am-959am
t1 = acc['HOUR'][acc['HOUR'].isin(
    [10, 11, 12, 13, 14, 15])].count()  # 1000am-0359pm
t2 = acc['HOUR'][acc['HOUR'].isin(
    [16, 17, 18])].count()  # 0400pm-0659pm
t3 = acc['HOUR'][acc['HOUR'].isin(
    [19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6])].count()  # 0700pm-0659am
print('Time: Count(Proportion)')
print('7:00 - 9:59 a.m.: %d(%.1f%%)' %
      (t0, t0/acc['HOUR'].count()*100))
print('10:00 a.m. - 3:59 p.m.: %d(%.1f%%)' %
      (t1, t1/acc['HOUR'].count()*100))
print('4:00 - 6:59 p.m.: %d(%.1f%%)' %
      (t2, t2/acc['HOUR'].count()*100))
print('7:00 p.m. - 6:59 a.m.: %d(%.1f%%)' %
      (t3, t3/acc['HOUR'].count()*100))

Time: Count(Proportion)
7:00 - 9:59 a.m.: 24693(15.5%)
10:00 a.m. - 3:59 p.m.: 53660(33.7%)
4:00 - 6:59 p.m.: 31329(19.7%)
7:00 p.m. - 6:59 a.m.: 49542(31.1%)


In [6]:
weekday_row = acc['WEEK_DAY'].isin([1, 2, 3, 4, 5])
weekday = acc['WEEK_DAY'][weekday_row].count()
weekend = acc['WEEK_DAY'][~weekday_row].count()
print('Day of week: Count(Proportion)')
print('Monday - Friday: %d(%.1f%%)' %
      (weekday, weekday/acc['WEEK_DAY'].count()*100))
print('Weekend: %d(%.1f%%)' %
      (weekend, weekend/acc['WEEK_DAY'].count()*100))

Day of week: Count(Proportion)
Monday - Friday: 111410(70.0%)
Weekend: 47814(30.0%)


In [7]:
r0 = acc['RAIN'][acc['RAIN'] == 1].count()
r1 = acc['RAIN'][acc['RAIN'] == 2].count()
r2 = acc['RAIN'][acc['RAIN'] == 3].count()
r3 = acc['RAIN'][acc['RAIN'].isin([9, 0])].count()

print('Rain: Count(Proportion)')
print('No Rain: %d(%.1f%%)' % (r0, r0/acc['RAIN'].count()*100))
print('Light Rain: %d(%.1f%%)' % (r1, r1/acc['RAIN'].count()*100))
print('Heavy Rain: %d(%.1f%%)' % (r2, r2/acc['RAIN'].count()*100))
print('Unknown: %d(%.1f%%)' % (r3, r3/acc['RAIN'].count()*100))

Rain: Count(Proportion)
No Rain: 136458(85.7%)
Light Rain: 18851(11.8%)
Heavy Rain: 3365(2.1%)
Unknown: 550(0.3%)


In [8]:
s0 = acc['SPEED_LMT'][acc['SPEED_LMT'] == 50].count()
s1 = acc['SPEED_LMT'][acc['SPEED_LMT'] < 50].count()
s2 = acc['SPEED_LMT'][acc['SPEED_LMT'] > 50].count()

print('Speed Limit: Count(Proportion)')
print('50km/h: %d(%.1f%%)' % (s0, s0/acc['SPEED_LMT'].count()*100))
print('Under 50km/h: %d(%.1f%%)' % (s1, s1/acc['SPEED_LMT'].count()*100))
print('Above 50km/h: %d(%.1f%%)' % (s2, s2/acc['SPEED_LMT'].count()*100))

Speed Limit: Count(Proportion)
50km/h: 135340(85.0%)
Under 50km/h: 3517(2.2%)
Above 50km/h: 20367(12.8%)


In [9]:
t0 = acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 3].count()
t1 = acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 1].count()
t2 = acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 2].count()
t3 = acc['TRAFF_CONG'][acc['TRAFF_CONG'].isin([9, 0])].count()

print('Traffic Congestion: Count(Proportion)')
print('None: %d(%.1f%%)' % (t0, t0/acc['TRAFF_CONG'].count()*100))
print('Severe Congestion: %d(%.1f%%)' % (t1, t1/acc['TRAFF_CONG'].count()*100))
print('Moderate Congestion: %d(%.1f%%)' % (t2, t2/acc['TRAFF_CONG'].count()*100))
print('Unknown: %d(%.1f%%)' % (t3, t3/acc['TRAFF_CONG'].count()*100))

Traffic Congestion: Count(Proportion)
None: 93839(58.9%)
Severe Congestion: 23799(14.9%)
Moderate Congestion: 38885(24.4%)
Unknown: 2701(1.7%)


In [10]:
j0 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 6].count()
j1 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 4].count()
j2 = acc['JCN_CTRL'][acc['JCN_CTRL'].isin([2, 3, 5])].count()
j3 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 1].count()
j4 = acc['JCN_CTRL'][acc['JCN_CTRL'] == 0].count()

print('Traffic Congestion: Count(Proportion)')
print('Non Junction: %d(%.1f%%)' % (j0, j0/acc['JCN_CTRL'].count()*100))
print('Traffic Signal: %d(%.1f%%)' % (j1, j1/acc['JCN_CTRL'].count()*100))
print('Other Control Types: %d(%.1f%%)' % (j2, j2/acc['JCN_CTRL'].count()*100))
print('No Control: %d(%.1f%%)' % (j3, j3/acc['JCN_CTRL'].count()*100))
print('Unknown: %d(%.1f%%)' % (j4, j4/acc['JCN_CTRL'].count()*100))

Traffic Congestion: Count(Proportion)
Non Junction: 120557(75.7%)
Traffic Signal: 21598(13.6%)
Other Control Types: 7580(4.8%)
No Control: 9415(5.9%)
Unknown: 74(0.0%)


In [11]:
r0 = acc['RD_TYPE'][acc['RD_TYPE'].isin([3, 4])].count()
r1 = acc['RD_TYPE'][acc['RD_TYPE'] == 1].count()
r2 = acc['RD_TYPE'][acc['RD_TYPE'] == 2].count()
r3 = acc['RD_TYPE'][acc['RD_TYPE'] == 0].count()

print('Traffic Congestion: Count(Proportion)')
print('Multi-/dual Carriageway: %d(%.1f%%)' % (r0, r0/acc['RD_TYPE'].count()*100))
print('One-way Carriageway: %d(%.1f%%)' % (r1, r1/acc['RD_TYPE'].count()*100))
print('Two-way Carriageway: %d(%.1f%%)' % (r2, r2/acc['RD_TYPE'].count()*100))
print('Unknown: %d(%.1f%%)' % (r3, r3/acc['RD_TYPE'].count()*100))

Traffic Congestion: Count(Proportion)
Multi-/dual Carriageway: 39048(24.5%)
One-way Carriageway: 63858(40.1%)
Two-way Carriageway: 56265(35.3%)
Unknown: 53(0.0%)


In [12]:
# split out the values of unknown
unknown_row = acc['CAS_AGE'].isin([0, 99, 999])
unknown = acc['CAS_AGE'][unknown_row]
dta_age = acc['CAS_AGE'][~unknown_row]

age_grouped = pd.cut(dta_age, [0, 15, 65, 120], labels=['<15', '15-65', '>65'])
age0 = age_grouped.value_counts()['<15']
age1 = age_grouped.value_counts()['15-65']
age2 = age_grouped.value_counts()['>65']
ageunknown = unknown.count()
print('Age: Count(Proportion)')
print('Under 15: %d(%.1f%%)' %
      (age0, age0/acc['CAS_AGE'].count()*100))
print('15 - 65: %d(%.1f%%)' %
      (age1, age1/acc['CAS_AGE'].count()*100))
print('Above 65: %d(%.1f%%)' %
      (age2, age2/acc['CAS_AGE'].count()*100))
print('Unknown: %d(%.1f%%)' %
      (ageunknown, ageunknown/acc['CAS_AGE'].count()*100))

Age: Count(Proportion)
Under 15: 10948(6.9%)
15 - 65: 132512(83.2%)
Above 65: 13812(8.7%)
Unknown: 1949(1.2%)


In [13]:
sex0 = acc['CAS_SEX'].value_counts()[1]
sex1 = acc['CAS_SEX'].value_counts()[2]
sex2 = acc['CAS_SEX'].value_counts()[9]
print('Sex: Count(Proportion)')
print('Male: %d(%.1f%%)' %
      (sex0, sex0/acc['CAS_SEX'].count()*100))
print('Famale: %d(%.1f%%)' %
      (sex1, sex1/acc['CAS_SEX'].count()*100))
print('Unknown: %d(%.1f%%)' %
      (sex2, sex2/acc['CAS_SEX'].count()*100))

Sex: Count(Proportion)
Male: 100390(63.0%)
Famale: 58570(36.8%)
Unknown: 264(0.2%)


In [14]:
injury0 = acc['INJURY'].value_counts()[1] + acc['INJURY'].value_counts()[2]
injury1 = acc['INJURY'].value_counts()[3]
print('Injury Severity: Count(Proportion)')
print('Killed or severe injury: %d(%.1f%%)' %
      (injury0, injury0/acc['INJURY'].count()*100))
print('Slight Injury: %d(%.1f%%)' %
      (injury1, injury1/acc['INJURY'].count()*100))

Injury Severity: Count(Proportion)
Killed or severe injury: 20575(12.9%)
Slight Injury: 138649(87.1%)


In [15]:
sb0 = acc['SB_WORN'][acc['SB_WORN'] == 1].count()
sb1 = acc['SB_WORN'][acc['SB_WORN'] == 2].count()
sb2 = acc['SB_WORN'][acc['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/acc['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/acc['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/acc['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 97637(61.3%)
No: 29325(18.4%)
Unknown: 32262(20.3%)


In [16]:
# Split into three group to check these variables proportion
accd = acc[acc['ROLE'] == 1]
accp = acc[acc['ROLE'] == 2]
accpd = acc[acc['ROLE'] == 3]

In [17]:
sb0 = accd['SB_WORN'][accd['SB_WORN'] == 1].count()
sb1 = accd['SB_WORN'][accd['SB_WORN'] == 2].count()
sb2 = accd['SB_WORN'][accd['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/accd['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/accd['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/accd['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 61077(80.4%)
No: 13735(18.1%)
Unknown: 1160(1.5%)


In [18]:
sb0 = accp['SB_WORN'][accp['SB_WORN'] == 1].count()
sb1 = accp['SB_WORN'][accp['SB_WORN'] == 2].count()
sb2 = accp['SB_WORN'][accp['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/accp['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/accp['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/accp['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 36560(68.3%)
No: 15590(29.1%)
Unknown: 1404(2.6%)


In [19]:
sb0 = accpd['SB_WORN'][accpd['SB_WORN'] == 1].count()
sb1 = accpd['SB_WORN'][accpd['SB_WORN'] == 2].count()
sb2 = accpd['SB_WORN'][accpd['SB_WORN'].isin([9, 0])].count()

print('Seat belt or crash helmet worn: Count(Proportion)')
print('Yes: %d(%.1f%%)' % (sb0, sb0/accpd['SB_WORN'].count()*100))
print('No: %d(%.1f%%)' % (sb1, sb1/accpd['SB_WORN'].count()*100))
print('Unknown: %d(%.1f%%)' % (sb2, sb2/accpd['SB_WORN'].count()*100))

Seat belt or crash helmet worn: Count(Proportion)
Yes: 0(0.0%)
No: 0(0.0%)
Unknown: 29698(100.0%)


In [20]:
s0 = acc['SEAT'][acc['SEAT'] == 3].count()
s1 = acc['SEAT'][acc['SEAT'] == 2].count()
s2 = acc['SEAT'][acc['SEAT'] == 1].count()
s3 = acc['SEAT'][acc['SEAT'].isin([4, 8])].count()
s4 = accd['SEAT'][accd['SEAT'].isin([6, 7])].count()
s5 = accd['SEAT'][accd['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/acc['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/acc['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/acc['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/acc['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/acc['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/acc['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 75972(47.7%)
Front Nearside: 9680(6.1%)
Rear: 32313(20.3%)
Standing: 9365(5.9%)
Others: 0(0.0%)
Unknown: 0(0.0%)


In [21]:
s0 = accd['SEAT'][accd['SEAT'] == 3].count()
s1 = accd['SEAT'][accd['SEAT'] == 2].count()
s2 = accd['SEAT'][accd['SEAT'] == 1].count()
s3 = accd['SEAT'][accd['SEAT'].isin([4, 8])].count()
s4 = accd['SEAT'][accd['SEAT'].isin([6, 7])].count()
s5 = accd['SEAT'][accd['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/accd['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/accd['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/accd['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/accd['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/accd['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/accd['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 75972(100.0%)
Front Nearside: 0(0.0%)
Rear: 0(0.0%)
Standing: 0(0.0%)
Others: 0(0.0%)
Unknown: 0(0.0%)


In [22]:
s0 = accp['SEAT'][accp['SEAT'] == 3].count()
s1 = accp['SEAT'][accp['SEAT'] == 2].count()
s2 = accp['SEAT'][accp['SEAT'] == 1].count()
s3 = accp['SEAT'][accp['SEAT'].isin([4, 8])].count()
s4 = accp['SEAT'][accp['SEAT'].isin([6, 7])].count()
s5 = accp['SEAT'][accp['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/accp['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/accp['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/accp['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/accp['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/accp['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/accp['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 0(0.0%)
Front Nearside: 9680(18.1%)
Rear: 32313(60.3%)
Standing: 9365(17.5%)
Others: 26(0.0%)
Unknown: 2170(4.1%)


In [23]:
s0 = accpd['SEAT'][accpd['SEAT'] == 3].count()
s1 = accpd['SEAT'][accpd['SEAT'] == 2].count()
s2 = accpd['SEAT'][accpd['SEAT'] == 1].count()
s3 = accpd['SEAT'][accpd['SEAT'].isin([4, 8])].count()
s4 = accpd['SEAT'][accpd['SEAT'].isin([6, 7])].count()
s5 = accpd['SEAT'][accpd['SEAT'].isin([0, 9])].count()

print('Seat Occupied: Count(Proportion)')
print('Driver/Rider: %d(%.1f%%)' % (s0, s0/accpd['SEAT'].count()*100))
print('Front Nearside: %d(%.1f%%)' % (s1, s1/accpd['SEAT'].count()*100))
print('Rear: %d(%.1f%%)' % (s2, s2/accpd['SEAT'].count()*100))
print('Standing: %d(%.1f%%)' % (s3, s3/accpd['SEAT'].count()*100))
print('Others: %d(%.1f%%)' % (s4, s4/accpd['SEAT'].count()*100))
print('Unknown: %d(%.1f%%)' % (s5, s5/accpd['SEAT'].count()*100))

Seat Occupied: Count(Proportion)
Driver/Rider: 0(0.0%)
Front Nearside: 0(0.0%)
Rear: 0(0.0%)
Standing: 0(0.0%)
Others: 0(0.0%)
Unknown: 29698(100.0%)


In [24]:
p0 = acc['PED_LOCATN'][acc['PED_LOCATN'] == 3].count()
p1 = acc['PED_LOCATN'][acc['PED_LOCATN'] == 4].count()
p2 = acc['PED_LOCATN'][acc['PED_LOCATN'].isin([1, 2, 5, 8])].count()
p3 = acc['PED_LOCATN'][acc['PED_LOCATN'].isin([0, 9])].count()

print('Pedestrian Location: Count(Proportion)')
print('On controlled crossing: %d(%.1f%%)' % (p0, p0/acc['PED_LOCATN'].count()*100))
print('Within 15M of controlled crossing: %d(%.1f%%)' % (p1, p1/acc['PED_LOCATN'].count()*100))
print('Others: %d(%.1f%%)' % (p2, p2/acc['PED_LOCATN'].count()*100))
print('Unknown: %d(%1.f%%)' % (p3, p3/acc['PED_LOCATN'].count()*100))

Pedestrian Location: Count(Proportion)
On controlled crossing: 4351(2.7%)
Within 15M of controlled crossing: 1853(1.2%)
Others: 22971(14.4%)
Unknown: 130049(82%)


In [25]:
pp0 = accpd['PED_LOCATN'][accpd['PED_LOCATN'] == 3].count()
pp1 = accpd['PED_LOCATN'][accpd['PED_LOCATN'] == 4].count()
pp2 = accpd['PED_LOCATN'][accpd['PED_LOCATN'].isin([1, 2, 5, 8])].count()
pp3 = accpd['PED_LOCATN'][accpd['PED_LOCATN'].isin([0, 9])].count()

print('Pedestrian Location: Count(Proportion)')
print('On controlled crossing: %d(%.1f%%)' % (pp0, pp0/accpd['PED_LOCATN'].count()*100))
print('Within 15M of controlled crossing: %d(%.1f%%)' % (pp1, pp1/accpd['PED_LOCATN'].count()*100))
print('Others: %d(%.1f%%)' % (pp2, pp2/accpd['PED_LOCATN'].count()*100))
print('Unkown: %d(%1.f%%)' % (pp3, pp3/accpd['PED_LOCATN'].count()*100))

Pedestrian Location: Count(Proportion)
On controlled crossing: 4351(14.7%)
Within 15M of controlled crossing: 1853(6.2%)
Others: 22971(77.3%)
Unkown: 523(2%)


In [26]:
print(acc['YEAR'].value_counts())
print(acc['YEAR'].value_counts(normalize=True))

2010    38248
2013    20596
2015    20381
2012    20210
2016    20132
2014    19854
2011    19803
Name: YEAR, dtype: int64
2010    0.240215
2013    0.129352
2015    0.128002
2012    0.126928
2016    0.126438
2014    0.124692
2011    0.124372
Name: YEAR, dtype: float64


In [29]:
# ACC_TIME
acc['ACC_TIME'] = acc['ACC_TIME'].map(
    lambda x: str_append(str(x)))
acc['HOUR'] = pd.to_datetime(acc['ACC_TIME'], format='%H%M').dt.hour
# Split into four group,
acc['HOUR'][acc['HOUR'].isin([7, 8, 9])] = '7:00 - 9:59 a.m.'
acc['HOUR'][acc['HOUR'].isin(
    [10, 11, 12, 13, 14, 15])] = '10:00 a.m. - 3:59 p.m.'
acc['HOUR'][acc['HOUR'].isin([16, 17, 18])] = '4:00 - 6:59 p.m.'
acc['HOUR'][acc['HOUR'].isin(
    [19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6])] = '7:00 p.m. - 6:59 a.m.'
# Convert to dummy variables
acc['HOUR'][acc['HOUR'] == '7:00 p.m. - 6:59 a.m.'] = 0
acc['HOUR'][acc['HOUR'] == '7:00 - 9:59 a.m.'] = 1
acc['HOUR'][acc['HOUR'] == '10:00 a.m. - 3:59 p.m.'] = 2
acc['HOUR'][acc['HOUR'] == '4:00 - 6:59 p.m.'] = 3
# Merge into acc
dummy = pd.get_dummies(acc['HOUR'], drop_first=True).rename(columns={1: '7:00 - 9:59 a.m.',
                                                                     2: '10:00 a.m. - 3:59 p.m.',
                                                                     3: '4:00 - 6:59 p.m.'})
acc = pd.concat([acc, dummy], axis=1).drop(['HOUR', 'ACC_TIME'], axis=1)

In [30]:
# Similarly, group and dummy WEEK_DAY,
# Split into two group, weekday and weekend, and Convert to dummy variables
acc['WEEK_DAY'][acc['WEEK_DAY'].isin([1, 2, 3, 4, 5])] = 1
acc['WEEK_DAY'][acc['WEEK_DAY'].isin([6, 7])] = 0
# Merge into acc
dummy = pd.get_dummies(acc['WEEK_DAY'], drop_first=True).rename(columns={1: 'Monday - Friday'})
acc = pd.concat([acc, dummy], axis=1).drop('WEEK_DAY', axis=1)

In [31]:
# Similarly, group and dummy RAIN
# Remove Unknown rows,
acc = acc[~acc['RAIN'].isin([9, 0])]
# Convert to dummy variables
acc['RAIN'][acc['RAIN'] == 1] = 0
acc['RAIN'][acc['RAIN'] == 2] = 1
acc['RAIN'][acc['RAIN'] == 3] = 2
# Merge into acc
dummy = pd.get_dummies(acc['RAIN'], drop_first=True).rename(columns={1: 'Light rain',
                                                                     2: 'Heavy rain'})
acc = pd.concat([acc, dummy], axis=1).drop('RAIN', axis=1)

In [32]:
# Similarly, group and dummy SPEED_LMT
# Convert to dummy variables
acc['SPEED_LMT'][acc['SPEED_LMT'] > 50] = 0
acc['SPEED_LMT'][acc['SPEED_LMT'] < 50] = 1
acc['SPEED_LMT'][acc['SPEED_LMT'] == 50] = 2
# Merge into acc
dummy = pd.get_dummies(acc['SPEED_LMT'], drop_first=True).rename(columns={1: 'Below 50 km/h',
                                                                          2: '50 km/h'})
acc = pd.concat([acc, dummy], axis=1).drop('SPEED_LMT', axis=1)

In [33]:
# Similarly, group and dummy TRAFF_CONG
# Remove Unknown rows,
acc = acc[~acc['TRAFF_CONG'].isin([9, 0])]
# Convert to dummy variables
acc['TRAFF_CONG'][acc['TRAFF_CONG'] == 3] = 0
# Merge into acc
dummy = pd.get_dummies(acc['TRAFF_CONG'], drop_first=True).rename(columns={1: 'Severe Congestion',
                                                                           2: 'Moderate Congestion'})
acc = pd.concat([acc, dummy], axis=1).drop('TRAFF_CONG', axis=1)

In [34]:
# Similarly, group and dummy JCN_CTRL
# Remove Unknown rows,
acc = acc[~acc['JCN_CTRL'].isin([9, 0])]
# Convert to dummy variables
acc['JCN_CTRL'][acc['JCN_CTRL'] == 6] = 0
acc['JCN_CTRL'][acc['JCN_CTRL'] == 4] = 1
acc['JCN_CTRL'][acc['JCN_CTRL'].isin([2, 3, 5])] = 2
acc['JCN_CTRL'][acc['JCN_CTRL'] == 1] = 3
# Merge into acc
dummy = pd.get_dummies(acc['JCN_CTRL'], drop_first=True).rename(columns={1: 'Non Junction',
                                                                         2: 'Traffic Signal',
                                                                         3: 'Other Control Types',
                                                                         4: 'No Control'})
acc = pd.concat([acc, dummy], axis=1).drop('JCN_CTRL', axis=1)

In [35]:
# Similarly, group and dummy RD_TYPE
# Remove Unknown rows
acc = acc[~acc['RD_TYPE'].isin([9, 0])]
# Convert to dummy variables
acc['RD_TYPE'][acc['RD_TYPE'].isin([3, 4])] = 0
# Merge into acc
dummy = pd.get_dummies(acc['RD_TYPE'], drop_first=True).rename(columns={1: 'One-way Carriageway',
                                                                        2: 'Two-way Carriageway'})
acc = pd.concat([acc, dummy], axis=1).drop('RD_TYPE', axis=1)

In [36]:
# Similarly, group and dummy CAS_AGE
# Remove Unknown rows
acc = acc[~acc['CAS_AGE'].isin([0, 99, 999])]
# Convert to dummy variables
acc['CAS_AGE'] = pd.cut(acc['CAS_AGE'], [0, 15, 66, 120], labels=[0, 1, 2])
# Merge into acc
dummy = pd.get_dummies(acc['CAS_AGE']).rename(columns={0: 'Under 15',
                                                       1: '15 - 65',
                                                       2: 'Above 65'})
acc = pd.concat([acc, dummy], axis=1).drop(['CAS_AGE', '15 - 65'], axis=1)

In [37]:
# Similarly, group and dummy CAS_SEX
# Remove Unknown rows
acc = acc[~acc['CAS_SEX'].isin([0, 9])]
# Convert to dummy variables
acc['CAS_SEX'][acc['CAS_SEX'] == 2] = 0
# Merge into acc
dummy = pd.get_dummies(acc['CAS_SEX'], drop_first=True).rename(columns={1: 'Male'})
acc = pd.concat([acc, dummy], axis=1).drop('CAS_SEX', axis=1)

In [38]:
# Similarly, group and dummy Year
dummy = pd.get_dummies(acc['YEAR'], prefix='YEAR').drop('YEAR_2016', axis=1)
acc = pd.concat([acc, dummy], axis=1).drop(['YEAR'], axis=1)

In [39]:
# Group INJURY for binary classification
acc['INJURY'][acc['INJURY'].isin([1, 2])] = 1
acc['INJURY'][acc['INJURY'] == 3] = 0
# which means the model would find the prob of seriously injury with the given conditions.

In [40]:
acc.head()

,INJURY,ROLE,SB_WORN,SEAT,PED_LOCATN,7:00 - 9:59 a.m.,10:00 a.m. - 3:59 p.m.,4:00 - 6:59 p.m.,Monday - Friday,Light rain,...,Two-way Carriageway,Under 15,Above 65,Male,YEAR_2010,YEAR_2011,YEAR_2012,YEAR_2013,YEAR_2014,YEAR_2015
0,1,1,1,3,0,0,0,0,1,0,...,1,0,0,1,1,0,0,0,0,0
1,0,1,2,3,0,0,1,0,1,0,...,1,0,0,1,1,0,0,0,0,0
2,0,1,1,3,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,2,1,2,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,0,1,1,3,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [41]:
acc.columns

Index(['INJURY', 'ROLE', 'SB_WORN', 'SEAT', 'PED_LOCATN', '7:00 - 9:59 a.m.',
       '10:00 a.m. - 3:59 p.m.', '4:00 - 6:59 p.m.', 'Monday - Friday',
       'Light rain', 'Heavy rain', '50 km/h', 'Severe Congestion',
       'Moderate Congestion', 'Traffic Signal', 'Other Control Types',
       'One-way Carriageway', 'Two-way Carriageway', 'Under 15', 'Above 65',
       'Male', 'YEAR_2010', 'YEAR_2011', 'YEAR_2012', 'YEAR_2013', 'YEAR_2014',
       'YEAR_2015'],
      dtype='object')

In [42]:
# split dataset to into 3 group, driver, passenger and pedestrian

In [43]:
acc_driver = acc[acc['ROLE'] == 1].drop(['ROLE', 'SEAT', 'PED_LOCATN'], axis=1)
acc_passen = acc[acc['ROLE'] == 2].drop(['ROLE', 'PED_LOCATN'], axis=1)
acc_pedest = acc[acc['ROLE'] == 3].drop(['ROLE', 'SEAT', 'SB_WORN'], axis=1)

In [44]:
# Similarly, group and dummy SB_WORN, but this is only for driver and passen
# Remove Unknown rows
acc_driver = acc_driver[~acc_driver['SB_WORN'].isin([0, 9])]
acc_passen = acc_passen[~acc_passen['SB_WORN'].isin([0, 9])]
# Convert to dummy variables
acc_driver['SB_WORN'][acc_driver['SB_WORN'] == 2] = 0
acc_passen['SB_WORN'][acc_passen['SB_WORN'] == 2] = 0
# Merge into acc
dummy = pd.get_dummies(acc_driver['SB_WORN'], drop_first=True).rename(columns={1: 'With belt or helmet'})
acc_driver = pd.concat([acc_driver, dummy], axis=1).drop('SB_WORN', axis=1)
dummy = pd.get_dummies(acc_passen['SB_WORN'], drop_first=True).rename(columns={1: 'With belt or helmet'})
acc_passen = pd.concat([acc_passen, dummy], axis=1).drop('SB_WORN', axis=1)

In [45]:
# Similarly, group and dummy SEAT, but this is only for passen
# Convert to dummy variables
acc_passen['SEAT'][acc_passen['SEAT'].isin([0, 6, 7, 9])] = 5
acc_passen['SEAT'][acc_passen['SEAT'].isin([4, 8])] = 4
acc_passen['SEAT'][acc_passen['SEAT'] == 3] = 0
# Merge into acc
# rmk Control variables is Driver/Rider
dummy = pd.get_dummies(acc_passen['SEAT'], drop_first=True).rename(
    columns={1: 'Rear', 2: 'Front nearside', 4: 'Standing', 5: 'Others'})
acc_passen = pd.concat([acc_passen, dummy], axis=1).drop('SEAT', axis=1)

In [46]:
# Similarly, group and dummy PED_LOCATN, but this is only for pedest
acc_pedest['PED_LOCATN'][acc_pedest['PED_LOCATN'].isin([0, 1, 2, 5, 8, 9])] = 0
acc_pedest['PED_LOCATN'][acc_pedest['PED_LOCATN'] == 3] = 1
acc_pedest['PED_LOCATN'][acc_pedest['PED_LOCATN'] == 4] = 2
# Merge into acc
# rmk Control variables is other
dummy = pd.get_dummies(acc_pedest['PED_LOCATN'], drop_first=True).rename(
    columns={1: 'On controlled crossing', 2: 'Within 15M of controlled crossing'})
acc_pedest = pd.concat([acc_pedest, dummy], axis=1).drop('PED_LOCATN', axis=1)

In [47]:
X_driver, y_driver = acc_driver.drop('INJURY', axis=1), acc_driver['INJURY']
X_passen, y_passen = acc_passen.drop('INJURY', axis=1), acc_passen['INJURY']
X_pedest, y_pedest = acc_pedest.drop('INJURY', axis=1), acc_pedest['INJURY']

In [48]:
X_driver = sm.add_constant(X_driver)
m_driver = sm.Logit(y_driver, X_driver)
r_driver = m_driver.fit()

Optimization terminated successfully.
         Current function value: 0.377748
         Iterations 6


In [49]:
r_driver.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                73331
Model:                          Logit   Df Residuals:                    73307
Method:                           MLE   Df Model:                           23
Date:                Sat, 10 Feb 2018   Pseudo R-squ.:                 0.02229
Time:                        22:53:10   Log-Likelihood:                -27701.
converged:                       True   LL-Null:                       -28332.
                                        LLR p-value:                1.179e-252
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -1.3304      0.062    -21.445      0.000      -1.452      -1.209
7:00 - 9:59 a.m.          -0.2248      0.036     -6.310      0.000      -0.295      -0.155
10:00 a.m. - 3:59 p.m.    -0.2161      0.027     -7.902      0.000      -0.270      -0.162
4:00 - 6:59 p.m.          -0.2447      0.032     -7.662      0.000      -0.307      -0.182
Monday - Friday           -0.0937      0.024     -3.961      0.000      -0.140      -0.047
Light rain                -0.1563      0.035     -4.423      0.000      -0.226      -0.087
Heavy rain                -0.1470      0.083     -1.769      0.077      -0.310       0.016
50 km/h                   -0.2007      0.031     -6.560      0.000      -0.261      -0.141
Severe Congestion         -0.1681      0.035     -4.869      0.000      -0.236      -0.100
Moderate Congestion       -0.1761      0.029     -6.040      0.000      -0.233      -0.119
Traffic Signal            -0.2763      0.058     -4.750      0.000      -0.390      -0.162
Other Control Types       -0.3622      0.037     -9.870      0.000      -0.434      -0.290
One-way Carriageway       -0.0702      0.031     -2.239      0.025      -0.132      -0.009
Two-way Carriageway        0.2637      0.031      8.533      0.000       0.203       0.324
Under 15                  -0.5129      0.069     -7.451      0.000      -0.648      -0.378
Above 65                   0.5197      0.064      8.126      0.000       0.394       0.645
Male                       0.2639      0.035      7.609      0.000       0.196       0.332
YEAR_2010                 -0.1458      0.039     -3.717      0.000      -0.223      -0.069
YEAR_2011                 -0.1260      0.045     -2.791      0.005      -0.214      -0.037
YEAR_2012                  0.0520      0.043      1.202      0.230      -0.033       0.137
YEAR_2013                  0.0809      0.043      1.889      0.059      -0.003       0.165
YEAR_2014                  0.1444      0.042      3.403      0.001       0.061       0.228
YEAR_2015                  0.0858      0.042      2.029      0.042       0.003       0.169
With belt or helmet       -0.4764      0.028    -16.847      0.000      -0.532      -0.421
==========================================================================================
"""

In [50]:
p_driver = r_driver.params
c_driver = r_driver.conf_int()
c_driver['OR'] = p_driver
c_driver.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(c_driver))

                            2.5%     97.5%  Odds Ratio
const                   0.234099  0.298549    0.264367
7:00 - 9:59 a.m.        0.744853  0.856459    0.798709
10:00 a.m. - 3:59 p.m.  0.763633  0.850029    0.805674
4:00 - 6:59 p.m.        0.735465  0.833533    0.782965
Monday - Friday         0.869337  0.953778    0.910579
Light rain              0.798082  0.916638    0.855308
Heavy rain              0.733561  1.015963    0.863291
50 km/h                 0.770584  0.868749    0.818196
Severe Congestion       0.789955  0.904435    0.845259
Moderate Congestion     0.791986  0.887867    0.838557
Traffic Signal          0.676868  0.850211    0.758604
Other Control Types     0.647865  0.748088    0.696176
One-way Carriageway     0.876638  0.991291    0.932204
Two-way Carriageway     1.225250  1.383058    1.301765
Under 15                0.523187  0.685246    0.598759
Above 65                1.483463  1.906179    1.681591
Male                    1.216402  1.393503    1.301945
YEAR_2010 

In [51]:
#X_passen = sm.add_constant(X_passen)
m_passen = sm.Logit(y_passen, X_passen)
r_passen = m_passen.fit()

Optimization terminated successfully.
         Current function value: 0.263662
         Iterations 8


In [52]:
r_passen.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                49881
Model:                          Logit   Df Residuals:                    49855
Method:                           MLE   Df Model:                           25
Date:                Sat, 10 Feb 2018   Pseudo R-squ.:                0.007390
Time:                        22:53:10   Log-Likelihood:                -13152.
converged:                       True   LL-Null:                       -13250.
                                        LLR p-value:                 1.945e-28
==========================================================================================
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
7:00 - 9:59 a.m.          -0.5421      0.053    -10.227      0.000      -0.646      -0.438
10:00 a.m. - 3:59 p.m.    -0.3842      0.040     -9.701      0.000      -0.462      -0.307
4:00 - 6:59 p.m.          -0.5230      0.050    -10.486      0.000      -0.621      -0.425
Monday - Friday           -0.3376      0.033    -10.090      0.000      -0.403      -0.272
Light rain                -0.1183      0.053     -2.237      0.025      -0.222      -0.015
Heavy rain                -0.5214      0.128     -4.070      0.000      -0.772      -0.270
50 km/h                   -0.6441      0.040    -16.220      0.000      -0.722      -0.566
Severe Congestion         -0.1397      0.048     -2.889      0.004      -0.235      -0.045
Moderate Congestion       -0.3907      0.041     -9.499      0.000      -0.471      -0.310
Traffic Signal            -0.0451      0.089     -0.505      0.614      -0.220       0.130
Other Control Types       -0.2646      0.047     -5.680      0.000      -0.356      -0.173
One-way Carriageway       -0.2945      0.040     -7.322      0.000      -0.373      -0.216
Two-way Carriageway       -0.0424      0.042     -0.999      0.318      -0.126       0.041
Under 15                  -1.3599      0.106    -12.849      0.000      -1.567      -1.152
Above 65                   0.7605      0.046     16.358      0.000       0.669       0.852
Male                      -0.1408      0.035     -3.977      0.000      -0.210      -0.071
YEAR_2010                 -0.8395      0.048    -17.351      0.000      -0.934      -0.745
YEAR_2011                 -0.7775      0.060    -13.035      0.000      -0.894      -0.661
YEAR_2012                 -0.7921      0.059    -13.408      0.000      -0.908      -0.676
YEAR_2013                 -0.7032      0.057    -12.333      0.000      -0.815      -0.591
YEAR_2014                 -0.6595      0.058    -11.424      0.000      -0.773      -0.546
YEAR_2015                 -0.5594      0.055    -10.135      0.000      -0.668      -0.451
With belt or helmet       -0.6465      0.037    -17.595      0.000      -0.719      -0.575
Front nearside            -0.0414      0.047     -0.874      0.382      -0.134       0.052
Standing                  -0.0992      0.047     -2.104      0.035      -0.192      -0.007
Others                    -0.3291      0.117     -2.807      0.005      -0.559      -0.099
==========================================================================================
"""

In [53]:
p_passen = r_passen.params
c_passen = r_passen.conf_int()
c_passen['OR'] = p_passen
c_passen.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(c_passen))

                            2.5%     97.5%  Odds Ratio
7:00 - 9:59 a.m.        0.524136  0.645188    0.581521
10:00 a.m. - 3:59 p.m.  0.630150  0.735972    0.681008
4:00 - 6:59 p.m.        0.537512  0.653587    0.592715
Monday - Friday         0.668161  0.761813    0.713452
Light rain              0.800940  0.985462    0.888423
Heavy rain              0.461874  0.763164    0.593705
50 km/h                 0.485829  0.567654    0.525150
Severe Congestion       0.790935  0.956049    0.869582
Moderate Congestion     0.624187  0.733390    0.676589
Traffic Signal          0.802296  1.138895    0.955893
Other Control Types     0.700528  0.840882    0.767503
One-way Carriageway     0.688464  0.806023    0.744928
Two-way Carriageway     0.882013  1.041590    0.958487
Under 15                0.208595  0.315856    0.256683
Above 65                1.953032  2.343439    2.139349
Male                    0.810477  0.931100    0.868697
YEAR_2010               0.392848  0.474890    0.431925
YEAR_2011 

In [54]:
#X_pedest = sm.add_constant(X_pedest)
m_pedest = sm.Logit(y_pedest, X_pedest)
r_pedest = m_pedest.fit()

Optimization terminated successfully.
         Current function value: 0.513965
         Iterations 6


In [55]:
r_pedest.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 INJURY   No. Observations:                28490
Model:                          Logit   Df Residuals:                    28466
Method:                           MLE   Df Model:                           23
Date:                Sat, 10 Feb 2018   Pseudo R-squ.:                 0.04356
Time:                        22:53:11   Log-Likelihood:                -14643.
converged:                       True   LL-Null:                       -15310.
                                        LLR p-value:                8.796e-268
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
7:00 - 9:59 a.m.                     -0.0928      0.045     -2.079      0.038      -0.180      -0.005
10:00 a.m. - 3:59 p.m.               -0.3193      0.037     -8.613      0.000      -0.392      -0.247
4:00 - 6:59 p.m.                     -0.3532      0.043     -8.217      0.000      -0.437      -0.269
Monday - Friday                      -0.0853      0.032     -2.687      0.007      -0.147      -0.023
Light rain                            0.0769      0.049      1.568      0.117      -0.019       0.173
Heavy rain                            0.3609      0.102      3.551      0.000       0.162       0.560
50 km/h                              -0.5922      0.050    -11.837      0.000      -0.690      -0.494
Severe Congestion                    -0.0038      0.041     -0.094      0.925      -0.084       0.076
Moderate Congestion                  -0.1978      0.035     -5.632      0.000      -0.267      -0.129
Traffic Signal                       -0.1899      0.067     -2.824      0.005      -0.322      -0.058
Other Control Types                  -0.1974      0.036     -5.511      0.000      -0.268      -0.127
One-way Carriageway                  -0.4576      0.040    -11.458      0.000      -0.536      -0.379
Two-way Carriageway                  -0.0812      0.041     -1.958      0.050      -0.163    8.88e-05
Under 15                             -0.0140      0.049     -0.284      0.777      -0.111       0.083
Above 65                              1.0235      0.035     29.516      0.000       0.956       1.092
Male                                 -0.2288      0.029     -8.006      0.000      -0.285      -0.173
YEAR_2010                            -0.3559      0.047     -7.633      0.000      -0.447      -0.265
YEAR_2011                            -0.2485      0.055     -4.536      0.000      -0.356      -0.141
YEAR_2012                            -0.2896      0.055     -5.270      0.000      -0.397      -0.182
YEAR_2013                            -0.1796      0.054     -3.308      0.001      -0.286      -0.073
YEAR_2014                            -0.2096      0.055     -3.792      0.000      -0.318      -0.101
YEAR_2015                            -0.1015      0.054     -1.866      0.062      -0.208       0.005
On controlled crossing                0.3794      0.041      9.328      0.000       0.300       0.459
Within 15M of controlled crossing     0.5449      0.056      9.709      0.000       0.435       0.655
=====================================================================================================
"""

In [56]:
p_pedest = r_pedest.params
c_pedest = r_pedest.conf_int()
c_pedest['OR'] = p_pedest
c_pedest.columns = ['2.5%', '97.5%', 'Odds Ratio']
print(np.exp(c_pedest))

                                       2.5%     97.5%  Odds Ratio
7:00 - 9:59 a.m.                   0.835113  0.994693    0.911417
10:00 a.m. - 3:59 p.m.             0.675696  0.781398    0.726627
4:00 - 6:59 p.m.                   0.645698  0.764195    0.702452
Monday - Friday                    0.862909  0.977200    0.918278
Light rain                         0.980956  1.188938    1.079951
Heavy rain                         1.175516  1.750879    1.434638
50 km/h                            0.501457  0.610098    0.553116
Severe Congestion                  0.919710  1.078971    0.996163
Moderate Congestion                0.765997  0.879026    0.820567
Traffic Signal                     0.724936  0.943579    0.827063
Other Control Types                0.765200  0.880559    0.820856
One-way Carriageway                0.585150  0.684314    0.632793
Two-way Carriageway                0.849962  1.000089    0.921975
Under 15                           0.895295  1.086152    0.986117
Above 65  